In [20]:
import numpy as np
from ppopt.mplp_program import MPLP_Program
from ppopt.mpmodel import MPModeler
from ppopt.mp_solvers.solve_mpqp import solve_mpqp, mpqp_algorithm
from numpy.polynomial.legendre import leggauss
from scipy.optimize import linprog
from typing import Union
from collections import defaultdict
from itertools import product
import time
import sympy as sp
from typing import Union, List
import pickle
# from sympy import symbols, expand, lambdify, Expr, simplify, pprint, Rel
import math
from pyomo.environ import *
import pyomo.environ as pyo
from typing import List
import itertools
import numpy as np
from IPython.display import display

In [21]:
def gauss_legendre_between_bounds(expr_coeffs: np.ndarray, n_gl: int, max_idx: int = 0, min_idx: int = 1):
    """
    Generate n Gauss–Legendre quadrature points and weights between min and max bounds
    defined by two linear expressions.

    Parameters:
        expr_coeffs (np.ndarray): 2xD array. Row 0 = max point co`efficients, Row 1 = min.
        n (int): Number of quadrature points.

    Returns:
        points (np.ndarray): (n, D) array of quadrature points.
        weights (np.ndarray): (n,) array of weights.
    """
    if expr_coeffs.shape[0] != 2:
        raise ValueError("expr_coeffs must have two rows")

    max_coeffs = expr_coeffs[max_idx]
    min_coeffs = expr_coeffs[min_idx]

    # Get Gauss–Legendre points and weights on [-1, 1]
    nodes, weights = leggauss(n_gl)
    weights = weights.reshape(-1,1)
    
    # Affine transformation to domain [min_coeffs, max_coeffs]
    points = 0.5 * (np.outer((nodes + 1), max_coeffs) + np.outer((1 - nodes), min_coeffs))

    # Adjust weights to match new domain
    weights = 0.5 * weights@(max_coeffs - min_coeffs).reshape(1,-1)

    return points, weights

In [22]:
def get_quadrature_points(solution, nq: int, t_vector: np.ndarray):
    # Augment t_vector once
    t_vector_aug = np.append(t_vector, 1).reshape(-1, 1)

    if isinstance(solution, list):
        qpoints, qweights = np.polynomial.legendre.leggauss(nq)
        min, max = solution[0], solution[1]
        qps_mapped = 0.5*(max*(1+qpoints) + min*(1-qpoints))
        qws_mapped = 0.5*(max-min)*qweights
        # print(max, min, qps_mapped, qws_mapped)
        return max, min, qps_mapped, qws_mapped

    for region in solution.critical_regions:
        if region.is_inside(t_vector):
            coeffs = np.concatenate([region.A, region.b], axis=1)[:2, :]
            qpoints, qweights = gauss_legendre_between_bounds(expr_coeffs=coeffs, n_gl=nq)
            return coeffs[0] @ t_vector_aug, coeffs[1] @ t_vector_aug, qpoints @ t_vector_aug, qweights @ t_vector_aug

    # print(f't_vector: {t_vector}')
    # print(f'solution:{solution}')
    raise ValueError("No region found that contains the given t_vector.")


In [23]:
def mpformulate_theta_bounds(flex_sol, num_theta:int, theta_bounds:list, num_design:int=0, design_bounds:list=None, psi_idx:int=0, theta_m:int=0):
    A0, b0, F0 = np.empty((len(flex_sol), num_theta)), np.empty((len(flex_sol), 1)), np.empty((len(flex_sol), num_design))
    num_cr = len(flex_sol.critical_regions)
    for i, region in enumerate(flex_sol.critical_regions):
        A0[i] = region.A[psi_idx,:num_theta]
        b0[i] = -region.b[psi_idx]
        F0[i] = -region.A[psi_idx, num_theta:num_theta+num_design]
    # print(f'num_cr:{num_cr}')
    # print(f'num_theta:{num_theta}')
    # print(f'num_design:{num_design}')
    # print(f"A0: {A0}")
    # print(f"b0: {b0}")
    # print(f"F0: {F0}")
    
    c = np.hstack([np.array([-1, 1]).reshape(1, -1), np.zeros((1, 2 * (num_theta - 1 - theta_m)))]).reshape(-1,1)
    # print(f'c:{c}')
    # print(f'c.shape: {c.shape}')
    
    row1_block = np.hstack([block for i in range(theta_m, num_theta) for block in (A0[:, [i]], np.zeros((num_cr, 1)))])
    row2_block = np.hstack([block for i in range(theta_m, num_theta) for block in (np.zeros((num_cr, 1)), A0[:, [i]])])
    bound_row = np.hstack([np.array([-1, 1]).reshape(1, -1), np.zeros((1, 2 * (num_theta - 1 - theta_m)))])
    A = np.vstack([row1_block, row2_block, bound_row, -np.eye(2*(num_theta-theta_m)), np.eye(2*(num_theta-theta_m))])
    # print(f'A: {A}')
    # print(f'A.shape: {A.shape}')
    
    x_lb = np.array([val for i in range(theta_m, len(theta_bounds)) for val in [theta_bounds[i][0]] * 2])
    x_ub = np.array([val for i in range(theta_m, len(theta_bounds)) for val in [theta_bounds[i][1]] * 2])
    b = np.vstack([b0, b0, np.zeros((1,1)), -x_lb.reshape(-1,1), x_ub.reshape(-1,1)])
    # print(f'b: {b}')
    # print(f'b.shape: {b.shape}')
    
    if F0.size==0 and theta_m==0:
        # print('here')
        return A, b, c, np.array([]), np.array([]), np.array([]), np.array([]) 
    
    F = np.vstack([F0, F0, np.zeros((1,num_design)), np.zeros((4*(num_theta-theta_m), num_design))]) if num_design>0 else np.vstack([F0, F0])
    # print(f'F:{F}')
    # print(f'F.shape: {F.shape}')
    if theta_m > 0:
        F_lltheta = np.hstack([A0[:, [i]] for i in range(theta_m)])
        # print(f'F_lltheta: {F_lltheta}')
        # print(f'F_lltheta.shape: {F_lltheta.shape}')
        F = np.hstack([np.vstack([-F_lltheta, -F_lltheta, np.zeros((1,len(range(theta_m)))), np.zeros((4*(num_theta-theta_m), theta_m))]), F]) if F.size > 0 else np.vstack([-F_lltheta, -F_lltheta, np.zeros((1,len(range(theta_m)))), np.zeros((4*(num_theta-theta_m), theta_m))])
    # print(f'F:{F}')
    # print(f'F.shape: {F.shape}')
    
    H = np.zeros((2*(num_theta-theta_m), theta_m+num_design))
    # print(f'H:{H}')
    # print(f'H.shape: {H.shape}')
    
    A_t = np.vstack([-np.eye(theta_m+num_design), np.eye(theta_m+num_design)])
    # print(f'A_t:{A_t}')
    # print(f'A_t.shape: {A_t.shape}')
    
    theta_lb = np.array([-theta_bounds[i][0] for i in range(theta_m)] + ([-j[0] for j in design_bounds] if isinstance(design_bounds, list) 
                                                                        else [])).reshape(-1, 1)
    theta_ub = np.array([theta_bounds[i][1] for i in range(theta_m)] + ([j[1] for j in design_bounds] if isinstance(design_bounds, list) 
                                                                        else [])).reshape(-1, 1)
    
    b_t = np.vstack([theta_lb, theta_ub])
    # print(f'b_t:{b_t}')
    # print(f'b_t.shape: {b_t.shape}')
    
    return A, b, c, H, A_t, b_t, F

In [24]:
def get_theta_bounds(flex_sol, numt, tbounds, numd:int=0, dbounds:list=None):
    
    theta_bound_dict = defaultdict(dict)
    prob_dict = defaultdict(dict)
    for i in range(numt):
        A, b, c, H, A_t, b_t, F = mpformulate_theta_bounds(flex_sol=flex_sol, num_theta=numt ,num_design=numd, theta_bounds=tbounds, design_bounds=dbounds, theta_m=i)
        if F.size != 0:
            prob = MPLP_Program(A=A, b=b, c=c, H=H, A_t=A_t, b_t=b_t, F=F)
            prob.process_constraints()
            solution = solve_mpqp(problem=prob, algorithm=mpqp_algorithm.geometric)
            prob_dict[f't{i}'] = prob
            theta_bound_dict[f't{i}'] = solution
        else:
            linsol = linprog(c=c, A_ub=A, b_ub=b)
            prob_dict[f't{i}'] = linsol
            theta_bound_dict[f't{i}'] = [linsol.x[1], linsol.x[0]]
            # if linsol.success:
                # print("Optimal value:", linsol.fun)
                # print("Optimal x:", linsol.x)
        print(f'Finished solving for theta{i+1}')
    probs = [p for key, p in prob_dict.items()]
    sols = [sol for key, sol in theta_bound_dict.items()]
    
    return probs, sols
    

In [25]:
def calculate_stocflexibility(sols, nq: Union[int, list], joint_func, d_vector: np.ndarray = None):
    
    # Validate nq if it's a list
    if isinstance(nq, list):
        if len(nq) != len(sols):
            raise ValueError("If nq is a list, it must have the same length as sols")

    def recurse(level: int, theta_prev: list, weight_prev: float) -> float:
        """
        Recursive inner function to compute nested quadrature.
        """
        # print(f'level:{level}')
        if level == len(sols):
            return weight_prev * joint_func(theta_prev)

        # Use nq[level] if nq is a list, otherwise use scalar nq
        nql = nq[level] if isinstance(nq, list) else nq

        t_vector = np.block([np.array(theta_prev), d_vector]) if isinstance(d_vector, np.ndarray) else np.array(theta_prev)
        # print(f't_vector:{t_vector}')
        # print(f'probs[{level}].A:{probs[level].A}')
        _, _, t_points, t_weights = get_quadrature_points(solution=sols[level], nq=nql, t_vector=t_vector)

        t_points = t_points.flatten()
        t_weights = t_weights.flatten()
        # print('t_points:', t_points)
        # print(f'theta_prev: {theta_prev}')
        return sum(recurse(level + 1, theta_prev + [v], weight_prev * w) for v, w in zip(t_points, t_weights))
    s = time.time()
    stflex =  recurse(level=0, theta_prev=[], weight_prev=1.0)
    e = time.time()
    print(f'Elapsed time for calculating sf index: {e- s}')
    
    return stflex
    

In [26]:
# Bansal (2000) Illustrative Example
t_bounds=[(0,4),(0,4)]
d_bounds=[(0,5), (0,5)]
nt = len(t_bounds)
nd = len(d_bounds)

m = MPModeler()

u = m.add_var(name='u')
x = m.add_var(name='x')
z = m.add_var(name='z')

t1 = m.add_param(name='t1')
t2 = m.add_param(name='t2')
d1 = m.add_param(name='d1')
d2 = m.add_param(name='d2')
m.add_constr(2*x - 3*z + t1 - d2 == 0)
m.add_constr(x - z/2 -t1/2 +t2/2 +d1 -7*d2/2 <= u)
m.add_constr(-2*x +2*z -4*t1/3 -t2 +2*d2 +1/3<= u)
m.add_constr(-x + 5*z/2 +t1/2 -t2 -d1 +d2/2 -1 <= u)
m.add_constr(-50 <= x)
m.add_constr(-50 <= z)
m.add_constr(t_bounds[0][0] <= t1)
m.add_constr(t_bounds[1][0] <= t2)
m.add_constr(d_bounds[0][0] <= d1)
m.add_constr(d_bounds[1][0] <= d2)
m.add_constr(t1 <= t_bounds[0][1])
m.add_constr(t2 <= t_bounds[1][1])
m.add_constr(d1 <= d_bounds[0][1])
m.add_constr(d2 <= d_bounds[1][1])
m.set_objective(u)
prob = m.formulate_problem()
prob.process_constraints()
solution_flexibility = solve_mpqp(problem=prob, algorithm=mpqp_algorithm.geometric)

start_time = time.time()
prob_list, sol_list = get_theta_bounds(flex_sol=solution_flexibility, numt=nt, numd=nd, tbounds=t_bounds, dbounds=d_bounds)
end_time = time.time()
print(f'Elapsed time for solving mp problems: {end_time-start_time}')

def joint_pdf(theta:list):
    return (2/np.pi)*np.exp(-2*((theta[0]-2)**2 + (theta[1]-2)**2))

Using a found active set [0, 2, 3]
Using a found active set [6, 9, 11, 12]
Finished solving for theta1
Using a found active set [6, 7]
Finished solving for theta2
Elapsed time for solving mp problems: 0.04101395606994629


In [27]:
# d_vector = np.array([5, 1.159054910657785])
# nq = 5
# sf_idx = calculate_stocflexibility(sols=sol_list, nq=nq, joint_func=joint_pdf, d_vector=d_vector)
# print(f'Stochastic Flexibility Index: {sf_idx:.4}')

## Finished Calculation of stochastic flexibility index

In [28]:
def get_bounds_regions(sols:List, min_idx:int=1, max_idx:int=0):
    theta_bounds_list = list()
    theta_regions_list = list()
    
    for theta_sol in sols:
        min_max_list = list()
        region_list = list()
        for cr in theta_sol.critical_regions:
            Ab = np.concatenate([cr.A, cr.b], axis=1)[:2]
            min_max_list.append([Ab[min_idx].tolist(), Ab[max_idx].tolist()])
    
            Ef = np.concatenate([cr.E, -cr.f], axis=1)
            region_list.append([row.tolist() for row in Ef])
    
        theta_bounds_list.append(np.array(min_max_list))
        theta_regions_list.append(np.array(region_list, dtype=object))  
    
    return theta_bounds_list, theta_regions_list

In [29]:
def generate_region_combos(region_sizes, n_gl):
    """Generate region index combinations based on critical region structure."""
    n_theta = len(region_sizes)
    region_combo_shape = []
    for k in range(n_theta):
        n_paths = int(np.prod(n_gl[:k])) if k > 0 else 1
        region_combo_shape.extend([range(region_sizes[k])] * n_paths)
    return list(itertools.product(*region_combo_shape))

In [30]:
def affine_expr(coeffs, symbols):
    return sum(c * s for c, s in zip(coeffs[:-1], symbols)) + coeffs[-1]

In [31]:
def normalized_lhs(ineq):
    return ineq.lhs.expand()

In [32]:
def compute_sf_exprs_regions(
    theta_bounds_list,
    theta_regions_list,
    joint_pdf_expr,
    d_syms,
    n_gl_list,
    theta_syms
):
    n_theta = len(theta_syms)
    quad_data = [np.polynomial.legendre.leggauss(n) for n in n_gl_list]
    region_sizes = [bounds.shape[0] for bounds in theta_bounds_list]
    region_combos = generate_region_combos(region_sizes, n_gl_list)

    sf_exprs = []
    sf_regions = []

    for region_combo in region_combos:
        combo_ptr = 0
        # Initialize integration paths: (theta_vals, weight, scale, constraints)
        paths = [([], 1, 1, [])]

        for level in range(n_theta):
            xi, wi = quad_data[level]
            new_paths = []

            for theta_vals, weight, scale, constraints in paths:
                region_idx = region_combo[combo_ptr]
                combo_ptr += 1

                bound_inputs = theta_vals + list(d_syms)
                bounds = theta_bounds_list[level][region_idx]
                t_min = affine_expr(bounds[0], bound_inputs)
                t_max = affine_expr(bounds[1], bound_inputs)

                # Get level-specific region constraints
                rows = theta_regions_list[level][region_idx]
                level_constraints = []
                for row in rows:
                    t_coeffs = row[:level]
                    d_coeffs = row[level:-1]
                    const = row[-1]
                    lhs = sum(c * theta_vals[i] for i, c in enumerate(t_coeffs)) + \
                          sum(c * d for c, d in zip(d_coeffs, d_syms)) + const
                    # level_constraints.append(sp.simplify(lhs <= 0))
                    level_constraints.append(lhs <= 0)

                new_constraints = constraints + level_constraints

                # Quadrature expansion for this level
                for q in range(len(xi)):
                    t = 0.5 * (t_max - t_min) * xi[q] + 0.5 * (t_max + t_min)
                    # new_theta_vals = theta_vals + [sp.simplify(t)]
                    new_theta_vals = theta_vals + [t]
                    new_weight = weight * wi[q]
                    new_scale = scale * 0.5 * (t_max - t_min)
                    new_paths.append((new_theta_vals, new_weight, new_scale, new_constraints))

            paths = new_paths

        # Final integration and region collection
        sf_sum = 0
        all_constraints = []
        for theta_vals, weight, scale, constraints in paths:
            theta_subs = {sym: val for sym, val in zip(theta_syms, theta_vals)}
            pdf_val = joint_pdf_expr.subs(theta_subs)
            sf_sum += weight * scale * pdf_val
            all_constraints.extend(constraints)

        # Deduplicate constraints symbolically
        unique_constraints = []
        for c in all_constraints:
            if not any(normalized_lhs(c) == normalized_lhs(u) and type(c) == type(u) for u in unique_constraints):
                unique_constraints.append(c)  # keep original form for clarity
            # if not any(is_same_inequality(c, u) for u in unique_constraints):
            #     unique_constraints.append(sp.simplify(c))

        sf_exprs.append(sf_sum)
        # sf_regions.append(sorted(all_constraints, key=str))
        # sf_exprs.append(sp.simplify(sf_sum))
        sf_regions.append(sorted(unique_constraints, key=str))

    return sf_exprs, sf_regions

In [33]:
def generate_constraints_from_expressions(
    expr_list: List[sp.Expr],
    region_list: List[List[sp.Expr]],
    instance: ConcreteModel,
    bounds_dict:dict,
    target: float = 1.0,
    big_m: float = 1e3
):
    """
    Adds constraint expressions to a Pyomo model using Big-M logic. For each expression in expr_list,
    the constraint is enforced only if the design variables lie in the corresponding critical region
    defined in region_list.

    Args:
        expr_list: List of SymPy expressions representing SF constraints.
        region_list: List of lists of SymPy inequality expressions defining valid regions.
        instance: Pyomo model (ConcreteModel).
        target: The minimum acceptable value for each SF expression.
        big_m: Big-M constant for constraint activation.
    """
    assert len(expr_list) == len(region_list), "Each expression must have a corresponding region definition."

    if not hasattr(instance, "generated_constraints"):
        instance.generated_constraints = ConstraintList()
    if not hasattr(instance, "region_constraints"):
        instance.region_constraints = ConstraintList()
    if not hasattr(instance, "region_binaries"):
        instance.region_binaries = Var(range(len(expr_list)), within=Binary)

    sf_expr_pyomo_list = list()

    for i, (sf_expr, region_exprs) in enumerate(zip(expr_list, region_list)):
        # Get all symbols in SF expression and region inequalities
        all_syms = sf_expr.free_symbols.union(*[reg.free_symbols for reg in region_exprs])
        all_syms = list(all_syms)

        # Ensure all symbols are added to the Pyomo model
        pyomo_vars = []
        for sym in all_syms:
            var_name = str(sym)
            if not hasattr(instance, var_name):
                setattr(instance, var_name, Var(bounds=bounds_dict[var_name]))
            pyomo_vars.append(getattr(instance, var_name))

        # Create a dict for substitution and lambdify
        sym_to_pyomo = {str(sym): getattr(instance, str(sym)) for sym in all_syms}
        lambdify_vars = list(sym_to_pyomo.keys())
        lambdify_vals = [sym_to_pyomo[s] for s in lambdify_vars]

        # Lambdify SF expression
        sf_func = sp.lambdify(lambdify_vars, sf_expr, modules=[{'exp': pyo.exp, 'pi': math.pi}, 'sympy'])
        sf_pyomo = sf_func(*lambdify_vals)
        sf_expr_pyomo_list.append(sf_pyomo)

        # Constraint: enforce SF ≥ target only when region binary = 1
        instance.generated_constraints.add(
            sf_pyomo >= target - big_m * (1 - instance.region_binaries[i])
        )

        # Region constraints: region_expr <= 0 + M*(1 - delta_i)
        for reg_expr in region_exprs:
            if not isinstance(reg_expr, sp.Rel):
                raise ValueError(f"Invalid region expression: {reg_expr} is not a relational (inequality) expression.")

            reg_func_lhs = sp.lambdify(lambdify_vars, reg_expr.lhs, modules='sympy')
            reg_func_rhs = sp.lambdify(lambdify_vars, reg_expr.rhs, modules='sympy')
            lhs_pyomo = reg_func_lhs(*lambdify_vals)
            rhs_pyomo = reg_func_rhs(*lambdify_vals)

            delta = instance.region_binaries[i]
            M_term = big_m * (1 - delta)

            if reg_expr.rel_op == '<=':
                instance.region_constraints.add(lhs_pyomo <= rhs_pyomo + M_term)
            elif reg_expr.rel_op == '<':
                instance.region_constraints.add(lhs_pyomo <= rhs_pyomo - 1e-6 + M_term)
            elif reg_expr.rel_op == '>=':
                instance.region_constraints.add(lhs_pyomo >= rhs_pyomo - M_term)
            elif reg_expr.rel_op == '>':
                instance.region_constraints.add(lhs_pyomo >= rhs_pyomo + 1e-6 - M_term)
            elif reg_expr.rel_op == '==':
                instance.region_constraints.add(lhs_pyomo >= rhs_pyomo - M_term)
                instance.region_constraints.add(lhs_pyomo <= rhs_pyomo + M_term)
            else:
                raise NotImplementedError(f"Unsupported relational operator: {reg_expr.rel_op}")

    # Optional: Only one region can be active
    instance.region_exclusivity = Constraint(expr=sum(instance.region_binaries[i] for i in range(len(expr_list))) == 1)
    if not hasattr(instance, 'sf'):
        instance.sf = Var(within=NonNegativeReals)

    if not hasattr(instance, 'sf_con'):
        instance.sf_con = Constraint(expr = instance.sf == sum(sf_expr_pyomo_list[i] * instance.region_binaries[i] for i in range(len(expr_list))))

In [34]:
t_bounds_list, t_regions_list = get_bounds_regions(sols=sol_list)

In [35]:
# --- Define Symbols ---
theta_syms = sp.symbols(f'theta_0:{nt}')
d_syms = sp.symbols(f'd0:{nd}')
theta_0, theta_1 = theta_syms
d1, d2 = d_syms

n_gl_list = [3,3]

# Bounds arrays
theta1_bounds_array = np.array([
    [[0.0, 0.0, 0.0], [0.0, 0.0, 4.0]],  # d0 ± 10
    [[0.75, -1.5, -1.25], [0.0, 0.0, 4.0]]  # d1 ± 5
])

theta2_bounds_array = np.array([
    [[-8/3, 2.0, -4.0, 2/3], [0.0, 0.0, 0.0, 4.0]],
    [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 4.0]],
    [[1/3, -0.5, 0.5, -1/3], [0.0, 0.0, 0.0, 4.0]]
])

# Critical regions (CRs)
# theta1_critical_regions: constraints in terms of d (shape: n_regions × n_ineqs × (m+1))
theta1_critical_regions = np.array([
    [[1, -2, -5/3]],
    [[-1, 2, 5/3]]
], dtype=object)

# theta2_critical_regions: constraints in terms of [theta1, d0, d1, const]
theta2_critical_regions = np.array([
    [[-8/3, 2, -4, -10/3], [8/3, -2, 4, -2/3]],
    [[8/3, -4.0, 4.0, -8/3], [-8/3, 2, -4, 2/3]],
    [[-8/3, 4, -4, 8/3]]
], dtype=object)

# --- Setup lists ---
theta_bounds_list = [theta1_bounds_array, theta2_bounds_array]
theta_regions_list = [theta1_critical_regions, theta2_critical_regions]

joint_pdf_expr = (2/sp.pi) * sp.exp(-2 * ((theta_0 - 2) ** 2 + (theta_1 - 2) ** 2))

In [36]:
theta_regions_list[0]

array([[[1, -2, -1.6666666666666667]],

       [[-1, 2, 1.6666666666666667]]], dtype=object)

In [37]:
t_regions_list[0]

array([list([[0.44721359549995826, -0.8944271909999157, -0.7453559924999327], [-1.0, -0.0, -0.0], [-0.0, -1.0, -0.0], [1.0, 0.0, -5.0], [0.0, 1.0, -5.0]]),
       list([[-0.4472135954999582, 0.8944271909999156, 0.7453559924999327], [-0.0, -1.0, -0.0], [1.0, 0.0, -5.0]])],
      dtype=object)

In [38]:
sf_exprs, sf_regions = compute_sf_exprs_regions(theta_bounds_list=t_bounds_list, theta_regions_list=t_regions_list,
                                                                              joint_pdf_expr=joint_pdf_expr, d_syms=[d1, d2], n_gl_list=n_gl_list,theta_syms=theta_syms)

AttributeError: 'BooleanTrue' object has no attribute 'lhs'

In [ ]:
print(f'Number of SF expressions: {len(sf_exprs)}')
print(f'Number of critical regions: {len(sf_regions)}')

In [ ]:
design_bounds = {f'd{i}':bounds for i, bounds in enumerate(d_bounds)}

In [ ]:
design_bounds